In [3]:
import pandas as pd
import numpy as np
from datetime import date

In [4]:
import Graph
import Path
import Evolutionary

Testing graph implementation

In [5]:
graph = Graph.get_graph()
assert len(graph.edges) == len(graph.vertices)

assert graph.get_neighbours('Czechia') == graph.get_neighbours('Czech_Republic')

assert np.array_equal(graph.get_stats('Czechia'), graph.get_stats('Czech_Republic'))

assert graph.get_active_cases("ABKHAZIA", date(2020, 3, 10)) == 0

assert graph.get_active_cases("Poland", date(2020, 3, 10)) == 17

assert graph.get_active_cases("Poland", date(2020, 3, 16)) == 122

assert graph.get_active_cases("Poland", date(2020, 3, 15)) == 102

assert graph.get_active_cases('POLAND', date(2019, 3, 10)) == 0

assert graph.get_active_cases('POLAND', date(2020, 3, 11)) - graph.get_active_cases('POLAND', date(2020, 3, 10)) == 5

Testing path

In [18]:
path = Path.Path('Poland', date(2020, 5, 1))

path.add('Czechia')
path.add('Germany')

assert path.countries == ['Poland', 'Czechia', 'Germany']

path.eval(graph)
score = graph.get_active_cases('Poland', date(2020, 5, 1)) + graph.get_active_cases(
    'Czechia', date(2020, 5, 2)) + graph.get_active_cases('Germany', date(2020, 5, 3))
assert score == path.score

path2 = Path.Path('Poland', date(2020, 5, 1))
#path2.add('Czechia')
path2.eval(graph)

assert path2.countries == ['Poland']

assert path2.score == 12253

assert graph.get_active_cases('Poland', date(2020, 5, 1)) == 12253

assert graph.get_active_cases('Czechia', date(2020, 5, 2)) == 7501

assert path2.get_profit(graph, 'Czechia') == 7501

Testing evolutionary algorithm

In [21]:
start_day = date(2020, 4, 24)
test = Evolutionary.EvolutionaryAlgorithm(graph, start_day, 100, 120, 0.05, 'roulette', 50, 'recovered')

best = test.find_path()

t = test.reproduce(test.random_lambda())

y = test.select_next_generation(t)

assert len(y) == len(test.active_generation)

In [34]:
def cross_paths(path1, path2):
    itrsct = [value for value in path1 if value in path2]
    point1, point2 = path1.index(itrsct[len(itrsct)//2]), path2.index(itrsct[len(itrsct)//2])
    need_len = len(path2)
    if(point1 == point2):
        ch1 = path1[:point1] + path2[point2:]
        ch2 = path2[:point2] + path1[point1:]
        return ch1, ch2
    if(point1 < point2):
        tmp, tmpp = path1, point1
        path1, point1 = path2, point2
        path2, point2 = tmp, tmpp
    path = path1[:point1] + path2[point2:]
    ch1 = path[:need_len]
    ch2 = path[-need_len:]
    return ch1, ch2
    

path1 = ['q', 'w', 'e', '[]', 'r', 't', 'y', 'u', 'i', 'o']
path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']

assert cross_paths(path1, path2) == (['q', 'w', 'e', '[]', 'f', 'g', 'h', 'j', 'k', 'l'],
 ['a', 's', 'd', '[]', 'r', 't', 'y', 'u', 'i', 'o'])

path1 = ['q', 'w', 'e', 'r', 't', 'y', '[]', 'u', 'i', 'o']
path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', '[]', 'f', 'g', 'h'],
 ['r', 't', 'y', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

path1 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', 'e', 'r', 't', 'y', '[]', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', '[]', 'f', 'g', 'h'],
 ['r', 't', 'y', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

path1 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', '[]', 'e', 'r', 't', 'y', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['a', 's', 'd', '[]', 'e', 'r', 't', 'y', 'u', 'i'],
 ['s', 'd', '[]', 'e', 'r', 't', 'y', 'u', 'i', 'o'])

path1 = ['a', '[]', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', 'e', 'r', 't', '[]', 'y', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', '[]', 's', 'd', 'f', 'g'],
 ['t', '[]', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l'])

path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path1 = ['q', 'w', 'e', 'r', 't', 'y', 'u', '[]', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', 'u', '[]', 'f', 'g'],
 ['t', 'y', 'u', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

# Measuring Evolutionary algorithm performance